# Flujos de Potencia Óptimos Lineales (DCOPF)

**Contribución de**: Uriel Sandoval

Esta notebook contiene la implementación de un flujos de potencia óptimos linealizados en el lenguaje Julia.

Para ejecutar los comandos los siguientes paquetes de Julia son necesarios:


* [JuMP](https://jump.dev/)  
* [OSQP](https://osqp.org/): Solver para problemas qudrácticos.
* [PowerSystems](https://nrel-siip.github.io/PowerSystems.jl/stable/): Paquete para leer archivos de MatPower y PSS/E.

Los paquetes pueden ser instalados de la siguiente forma:

```Julia
import Pkg;
Pkg.add(["JuMP", "OSQP", "PowerSystems"])
```

Los archivos de prueba fueron obtenidos del repositorio de [Power Grid Lib](https://github.com/power-grid-lib/pglib-opf), el cual es mantenido activamente por el [IEEE PES Task Force](https://arxiv.org/abs/1908.02788).

En la literatura se pueden encontrar diversas formulaciones del DCOPF, sin embargo para propósitos educativos y tratando ser lo más claro posible, la formulación aquí mostrada es igual a la implementada en [Matpower](https://matpower.org/).

Adicionalmente, el lector puede verificar sus resultados con los reportados en [The Power Grid Library for Benchmarking AC Optimal Power Flow Algorithms](https://arxiv.org/abs/1908.02788).


Empezamos importando los paquetes necesarios

In [1]:
using JuMP, OSQP;
using PowerSystems;

A continuación usamos la librería PowerSystems para leer el archivo *pglib_opf_case5_pjm.m*

In [2]:
# Leemos los datos utilizando PowerSystems
dat = PowerSystems.PowerModelsData("data/pglib_opf_case5_pjm.m").data;

# Los siguientes únicamente son aliases
branches = dat["branch"];
buses = dat["bus"];
gens = dat["gen"];
loads = dat["load"];
shunts = dat["shunt"];

┌ Info: extending matpower format with data: areas 1x3
└ @ PowerSystems C:\Users\UFSANDOVAL\.julia\packages\PowerSystems\hE7SB\src\parsers\pm_io\matpower.jl:332
┌ Info: removing 1 cost terms from generator 4: [4000.0, 0.0]
└ @ PowerSystems C:\Users\UFSANDOVAL\.julia\packages\PowerSystems\hE7SB\src\parsers\pm_io\data.jl:1867
┌ Info: removing 1 cost terms from generator 1: [1400.0, 0.0]
└ @ PowerSystems C:\Users\UFSANDOVAL\.julia\packages\PowerSystems\hE7SB\src\parsers\pm_io\data.jl:1867
┌ Info: removing 1 cost terms from generator 5: [1000.0, 0.0]
└ @ PowerSystems C:\Users\UFSANDOVAL\.julia\packages\PowerSystems\hE7SB\src\parsers\pm_io\data.jl:1867
┌ Info: removing 1 cost terms from generator 2: [1500.0, 0.0]
└ @ PowerSystems C:\Users\UFSANDOVAL\.julia\packages\PowerSystems\hE7SB\src\parsers\pm_io\data.jl:1867
┌ Info: removing 1 cost terms from generator 3: [3000.0, 0.0]
└ @ PowerSystems C:\Users\UFSANDOVAL\.julia\packages\PowerSystems\hE7SB\src\parsers\pm_io\data.jl:1867


In [3]:
# Creamos los conjuntos a ser utilizados en los indices de JuMP
B = keys(buses);
L = keys(branches);
G = keys(gens);
C = keys(loads);
S = keys(shunts);

@show(B)
@show(L)

##  Creamos un mapeo  bus -> elemento, para los dispositivos de una terminal

# Mapeo: bus -> [cargas]
load_map = Dict((i, String[]) for i in B);
for (key, load) in loads
    push!(load_map[string(load["load_bus"])], key)
end

# Mapeo: bus -> [gens]
gen_map = Dict((i, String[]) for i in keys(buses));
for (key, load) in gens
    push!(gen_map[string(load["gen_bus"])], key)
end

# Mapeo: bus -> [shunts]
shunt_map = Dict((i, String[]) for i in keys(buses));
for (key, load) in shunts
    push!(shunt_map[string(load["shunt_bus"])], key)
end

B = ["4", "1", "5", "2", "3"]
L = ["4", "1", "5", "2", "6", "3"]


## Modelado de la red eléctrica

Esta sección presenta el modelado de la red eléctrica utilizando JuMP. Se consideran los elementos más importantes de un sistemas de potencia:

* Nodos.
* Ramas (lineas de transmisión y transformadores).
* Cargas y equipos de compensación shunt.
* Generadores




In [4]:
# Creamos el modelo de JuMP y asignamos el solver a utilizar (OSQP)
mod = Model(OSQP.Optimizer);

A diferencia del ACOPF, en esta formulación únicamente consideramos los ángulos de voltaje nodales:

In [5]:
@variable(mod, δ[k in B], start = 0.0);
for (k, bus_data) in buses
    
    # Referencia angular para el nodo "slack"
    if (bus_data["bus_type"] == 3)
        set_lower_bound(δ[k], bus_data["va"]);
        set_upper_bound(δ[k], bus_data["va"]);
    end
end

Una aproximación lineal al flujo a través de los elementos de la red de transmisión:

$$
P_{km} = \frac{\delta_k - \delta_m}{x_s \tau}
$$


In [6]:
Pinj_f_bra = Dict((k, []) for k in B);
Pinj_t_bra = Dict((k, []) for k in B);

for brn in values(branches)
    if (brn["br_status"] == 0)
        continue
    end
    # Nodos k y m
    k = string(brn["f_bus"]);
    m = string(brn["t_bus"]);

    Pkm = @expression(mod, (δ[k] - δ[m]) / (brn["br_x"] * brn["tap"]));

    push!(Pinj_f_bra[k], Pkm);
    push!(Pinj_t_bra[m], -Pkm);

    if (brn["rate_a"] > 0)
        # En caso que sea necesario vigilar el flujo en la rama
        @constraint(mod, -brn["rate_a"] <= Pkm <= brn["rate_a"]);
    end

end

In [7]:
@variable(mod, gens[i]["pmin"] <= Pg[i in G] <= gens[i]["pmax"]);

### Ecuación de balance de potencia activa

$$
\sum_{i \in \mathcal{G}} p_{G_i} - \sum_{i \in \mathcal{C}} p_{L_i} - 
\sum_{(k,m) \in \mathcal{E}} p_{km}  -
\sum_{(m,k) \in \mathcal{E}} p_{mk} = 0 \quad \forall k \in \mathcal{B} 
$$

In [8]:
@constraint(mod, dP[k in B],sum(Pg[i] for i in gen_map[k]) -
                            sum(loads[load]["pd"] for load in load_map[k]) -
                            sum(Pkm for Pkm in Pinj_f_bra[k]) -
                            sum(Pmk for Pmk in Pinj_t_bra[k]) ==0);


### Función objetivo

La función objetivo del problema de flujos óptimos es minimizar el costo de producción total:

$$
\min \sum_{i \in \mathcal{G}} f_{C_i}\left( P_{G_i} \right) = \sum_{i \in \mathcal{G}}\left( c_i P_{G_i}^2 + b_i P_{G_i} + c_i \right)
$$


In [9]:
@objective(mod, Min, 
    sum(sum(gen["cost"][j] * Pg[i] ^(gen["ncost"] - j) for j in 1:gen["ncost"]) for (i, gen) in gens));

In [10]:
mod

A JuMP Model
Minimization problem with:
Variables: 10
Objective function type: GenericQuadExpr{Float64,VariableRef}
`GenericAffExpr{Float64,VariableRef}`-in-`MathOptInterface.EqualTo{Float64}`: 5 constraints
`GenericAffExpr{Float64,VariableRef}`-in-`MathOptInterface.Interval{Float64}`: 6 constraints
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 6 constraints
`VariableRef`-in-`MathOptInterface.LessThan{Float64}`: 6 constraints
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: OSQP
Names registered in the model: Pg, dP, δ

In [11]:
optimize!(mod)

┌ Warning: MathOptInterface.VariablePrimalStart() is not supported by MathOptInterface.Bridges.LazyBridgeOptimizer{MathOptInterface.Utilities.CachingOptimizer{OSQP.MathOptInterfaceOSQP.Optimizer,MathOptInterface.Utilities.UniversalFallback{MathOptInterface.Utilities.Model{Float64}}}}. This information will be discarded.
└ @ MathOptInterface.Utilities C:\Users\UFSANDOVAL\.julia\packages\MathOptInterface\k7UUH\src\Utilities\copy.jl:185


-----------------------------------------------------------------
           OSQP v0.6.0  -  Operator Splitting QP Solver
              (c) Bartolomeo Stellato,  Goran Banjac
        University of Oxford  -  Stanford University 2019
-----------------------------------------------------------------
problem:  variables n = 10, constraints m = 23
          nnz(P) + nnz(A) = 46
settings: linear system solver = qdldl,
          eps_abs = 1.0e-003, eps_rel = 1.0e-003,
          eps_prim_inf = 1.0e-004, eps_dual_inf = 1.0e-004,
          rho = 1.00e-001 (adaptive),
          sigma = 1.00e-006, alpha = 1.60, max_iter = 4000
          check_termination: on (interval 25),
          scaling: on, scaled_termination: off
          warm start: on, polish: off, time_limit: off

iter  objective    pri res    dua res    rho        time
   1 -2.3870e+004  4.68e+000  1.28e+006  1.00e-001  6.53e-005s
 200  1.7724e+004  3.71e-002  4.15e+002  8.00e-001  1.61e-004s
 400  1.9043e+004  2.51e-001  2.03e+001  1.

Podemos obtener los valores resultantes y actualizar nuestro diccionario de datos

In [12]:
print(termination_status(mod))
if termination_status(mod) in (MOI.OPTIMAL, MOI.LOCALLY_SOLVED)
    @info("Problema resuelto. Actualizando variables")
    println("Función objetivo ", objective_value(mod));
    for (k, bus) in buses
        bus["va"] = value.(δ[k]);
    end
    for (i, gen) in gens
        gen["pg"] = value.(Pg[i]);
    end
end

OPTIMAL

┌ Info: Problema resuelto. Actualizando variables
└ @ Main In[12]:3


Función objetivo 17491.70550869146
